## **Install dependencies**

In [1]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

bash: line 1: apt-get: command not found
bash: line 2: apt-get: command not found


In [2]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [3]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Processing /tmp/mecab-python-0.996
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for mecab-python: filename=mecab_python-0.996_ko_0.9.2-cp38-cp38-macosx_11_0_arm64.whl size=5166 sha256=24226842f41b68a4f68ead1f4f452ae40a4c89fdbf839ca75c05e8e68becaaa3
  Stored in directory: /Users/onmywave/Library/Caches/pip/wheels/e3/88/e7/a947778cce3c142d5721c0629e05db7b09979d3a973277ec2f


Failed to build mecab-python
  Attempting uninstall: mecab-python
    Found existing installation: mecab-python 0.996-ko-0.9.2
    Uninstalling mecab-python-0.996-ko-0.9.2:
      Successfully uninstalled mecab-python-0.996-ko-0.9.2
  Running setup.py install for mecab-python: started
  Running setup.py install for mecab-python: finished with status 'done'


  DEPRECATION: mecab-python was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. Discussion can be found at https://github.com/pypa/pip/issues/8368


## **Example**

In [1]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab
from tqdm import tqdm
import re
import pickle
import pandas as pd
from collections import defaultdict, Counter

In [2]:


# -*- coding: utf-8 -*-



def clean_text(text):
    """
    한글, 영문, 숫자만 남기고 제거한다.
    :param text:
    :return:
    
    """
    text = text.replace(".", " ").strip()
    text = text.replace("·", " ").strip()
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+'
    text = re.sub(pattern=pattern, repl='', string=text)
    return text


def get_nouns(tokenizer, sentence):
    """
    단어의 길이가 2이상인 일반명사(NNG), 고유명사(NNP), 외국어(SL)만을 반환한다.
    :param tokenizer:
    :param sentence:
    :return:
    """
    tagged = tokenizer.pos(sentence)
    nouns = [s for s, t in tagged if t in ['SL', 'NNG', 'NNP'] and len(s) > 1]
    return nouns


def delete_stopwords(processed_data):
    """
    불용어 처리
    """
    data = []
    stopwords = ['고려','대학교','안녕','경우','학교','교환','과목','나라','시간','한국','정도','절차',
                 '작성','신청','대부분','경험','사용','학생','생각','진행','가능','관련','확인']
    for doc in processed_data:
      word_list = [x for x in doc if x not in stopwords]
      data.append(word_list)

    return data


def tokenize(i,df, tokenization):
    tokenizer = tokenization()
    processed_data = []
    
    for sent in tqdm(df[i]):
        sentence = clean_text(sent.replace('\n', '').strip())
        processed_data.append(get_nouns(tokenizer, sentence))
    return processed_data

def get_nouns_for_okt(tokenizer, sentence):
    """
    okt를 위한 method
    :return:
    noun
    """
    tagged = tokenizer.pos(sentence)
    nouns = [s for s, t in tagged if t =='Noun' and len(s) > 1]
    return nouns
  
def tokenize_for_okt(i,df, tokenization):
    tokenizer = Okt()
    processed_data = []
    
    for sent in tqdm(df[i]):
        sentence = clean_text(sent.replace('\n', '').strip())
        processed_data.append(get_nouns_for_okt(tokenizer, sentence))
    return processed_data


          
if __name__ == '__main__':
    
    data = pd.read_excel('data.xlsx')
    word_list = lambda x : [word for sublist in x for word in sublist ]
    words = []

    processed_data1 = []
    processed_data2 = []
    processed_data3 = []

    for i in range(1,10):
      data1 = tokenize(i,data,Mecab) 
      data1 = delete_stopwords(processed_data1)
      processed_data1.append(word_list(data1))

      data2 = tokenize(i,data,Kkma) 
      data2 = delete_stopwords(processed_data2)
      processed_data2.append(word_list(data2))

      data3 = tokenize_for_okt(i,data,Okt) 
      data3 = delete_stopwords(processed_data3)
      processed_data3.append(word_list(data3))


100%|██████████| 1200/1200 [00:02<00:00, 568.01it/s]


[0.016s][warning][os,thread] Attempt to protect stack guard pages failed (0x0000000169aac000-0x0000000169ab8000).
[0.016s][warning][os,thread] Attempt to deallocate stack guard pages failed.


  7%|▋         | 83/1200 [00:21<02:46,  6.71it/s] 

In [ ]:
import pickle
with open('list1.txt', 'wb') as f:
  pickle.dump(processed_data1, f)
with open('list2.txt', 'wb') as f:
  pickle.dump(processed_data2, f)
with open('list3.txt', 'wb') as f:
  pickle.dump(processed_data3, f)

In [6]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

RuntimeError: Compiled extensions are unavailable. If you've installed from a package, ask the package maintainer to include compiled extensions. If you're building Gensim from source yourself, install Cython and a C compiler, and then run `python setup.py build_ext --inplace` to retry. 

In [ ]:
Mecab_nouns = word_list(processed_data1)
Kkma_nouns = word_list(processed_data2)
Okt_nouns = word_list(processed_data3)

print(processed_data3)

In [ ]:
!pip install pyLDAvis

In [ ]:
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt


In [ ]:
# Mecab, Kkma, Okt 비교
Mecab_Counter = Counter(Mecab_nouns)
Kkma_Counter = Counter(Kkma_nouns)
Okt_Counter = Counter(Okt_nouns)

In [ ]:
print(Mecab_Counter)
print(Kkma_Counter)
print(Okt_Counter)

In [ ]:
dictionary = corpora.Dictionary(processed_data1)
dictionary.filter_extremes(no_below=2, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in processed_data1]

In [ ]:
num_topics = 8
chunksize = 2000
passes = 8
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [ ]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

In [ ]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'f.html')